In [1]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

# Cargamos solo el 10% de los datos de entrenamiento para reducir el tiempo de entrenamiento
train_data = pd.read_csv('train.csv').sample(frac=0.1, random_state=42)
test_data = pd.read_csv('test.csv')  # Cargar el conjunto completo de prueba

# Imputación de valores faltantes en columnas categóricas
categorical_columns = [
    'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA',
    'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE', 'ESTU_PAGOMATRICULAPROPIO'
]

for column in categorical_columns:
    # Rellenamos valores faltantes en el conjunto de entrenamiento y prueba con la moda del conjunto de entrenamiento
    mode_value = train_data[column].mode()[0]
    train_data[column] = train_data[column].fillna(mode_value)
    if column in test_data.columns:
        test_data[column] = test_data[column].fillna(mode_value)

# Definimos las características (X) y la variable objetivo (y) para el conjunto de entrenamiento reducido
X_train = train_data.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'])
y_train = train_data['RENDIMIENTO_GLOBAL']

# Definimos el conjunto de características (X_test) para el conjunto de prueba
X_test = test_data.drop(columns=['ID'])

# Identificamos las columnas numéricas y categóricas para el pipeline
numeric_features = ['PERIODO']
categorical_features = categorical_columns

# Creamos transformadores para las columnas numéricas y categóricas
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first', handle_unknown='ignore')

# Creamos un preprocesador que aplique las transformaciones correspondientes
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Creamos un pipeline que incluya el preprocesador y el modelo SVM
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(kernel='linear', random_state=42))
])

# Entrenamos el modelo en el conjunto reducido de entrenamiento
pipeline.fit(X_train, y_train)

# Realizamos predicciones en el conjunto de prueba completo
all_predictions = pipeline.predict(X_test)

# Creamos el DataFrame para el archivo qué subiremos a Kaggle
submission = pd.DataFrame({
    'ID': test_data['ID'],  # Usar los IDs del conjunto de prueba completo
    'RENDIMIENTO_GLOBAL': all_predictions
})

# Guardamos el archivo de envío en formato csv
submission.to_csv('solucion.csv', index=False)
print("El archivo 'solución.csv' ha sido generado y está listo para descargar.")

El archivo 'solución.csv' ha sido generado y está listo para descargar.
